In [8]:
import os
import pyvtk
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

In [9]:
def read_element_output(data_dir, load_wave_data=True):
    ################ open files ################
    # filenames
    nc_fnames = [f for f in os.listdir(data_dir) if 'axisem3d_synthetics.nc' in f]
    print('files to open: ', nc_fnames)

    # open files
    nc_files = []
    for nc_fname in nc_fnames:
        nc_files.append(xr.open_dataset(data_dir + '/' + nc_fname))

    ################ variables that are the same in the datasets ################
    # read Na grid (all azimuthal dimensions)
    na_grid = nc_files[0].data_vars['list_na_grid'].values.astype(int)

    # read time
    data_time = nc_files[0].data_vars['data_time'].values

    ################ variables to be concatenated over the datasets ################
    # define empty lists of xarray.DataArray objects
    xda_list_element_na = []
    xda_list_element_coords = []
    dict_xda_list_element = {}
    dict_xda_data_wave = {}
    for nag in na_grid:
        dict_xda_list_element[nag] = []
        dict_xda_data_wave[nag] = []

    # loop over nc files
    for nc_file in nc_files:
        # append DataArrays
        xda_list_element_na.append(nc_file.data_vars['list_element_na'])
        xda_list_element_coords.append(nc_file.data_vars['list_element_coords'])
        for nag in na_grid:
            dict_xda_list_element[nag].append(nc_file.data_vars['list_element__NaG=%d' % nag])
            dict_xda_data_wave[nag].append(nc_file.data_vars['data_wave__NaG=%d' % nag])

    # concat xarray.DataArray
    xda_list_element_na = xr.concat(xda_list_element_na, dim='dim_element')
    xda_list_element_coords = xr.concat(xda_list_element_coords, dim='dim_element')
    for nag in na_grid:
        dict_xda_list_element[nag] = xr.concat(dict_xda_list_element[nag], dim='dim_element__NaG=%d' % nag)
        dict_xda_data_wave[nag] = xr.concat(dict_xda_data_wave[nag], dim='dim_element__NaG=%d' % nag)

    # read data to numpy.ndarray
    list_element_na = xda_list_element_na.values.astype(int)
    list_element_coords = xda_list_element_coords.values
    dict_list_element = {}
    dict_data_wave = {}
    for nag in na_grid:
        dict_list_element[nag] = dict_xda_list_element[nag].values.astype(int)
        if load_wave_data:
            dict_data_wave[nag] = dict_xda_data_wave[nag].values

    ############### return ################
    if load_wave_data:
        return na_grid, data_time, list_element_na, list_element_coords, dict_list_element, dict_data_wave
    else:
        return na_grid, data_time, list_element_na, list_element_coords, dict_list_element, dict_xda_data_wave

In [3]:
len([0.        , 0.17453293, 0.34906585, 0.52359878, 0.6981317 , 0.87266463, 1.04719755, 1.22173048, 1.3962634 , 1.57079633, 1.74532925, 1.91986218, 2.0943951 , 2.26892803, 2.44346095, 2.61799388, 2.7925268 , 2.96705973, 3.14159265, 3.31612558, 3.4906585 , 3.66519143, 3.83972435, 4.01425728, 4.1887902 , 4.36332313, 4.53785606, 4.71238898, 4.88692191, 5.06145483, 5.23598776, 5.41052068, 5.58505361, 5.75958653, 5.93411946, 6.10865238])

36

In [10]:
ModelNameList = []
for imodel in np.arange(0,1):
    ModeName = 'SolidSphereModel%04dRTZ' %imodel
    ModelNameList.append(ModeName)

for ModelName in ModelNameList:
    print('Processing ', ModelName)
    # data dir
    data_dir = '../Runs/%s/output/elements/orthogonal_azimuthal_slices' %ModelName

    # read
    na_grid, data_time, list_element_na, list_element_coords, \
    dict_list_element, dict_data_wave = read_element_output(data_dir)

    # wave dimension to animation
    wave_channel = 'z'
    wave_dim = 'spz'.index(wave_channel)

    # time steps
    ntime = len(data_time)

    # phi of the slices
    phi_slices = np.radians(np.arange(0,360,400))
    nslice = len(phi_slices)

    # GLL coords on elementsnslice
    nelem = list_element_coords.shape[0]
    ngll = list_element_coords.shape[1]
    # flattened coords, (s, z)
    element_coords_sz = list_element_coords.reshape((nelem * ngll), 2)

    # connectivity list, shared by all slices
    # with GLL_points_one_edge = [0, 2, 4] in the inparam.output.yaml,
    # the GLL point layout on each element is
    # 0--1
    # |  |
    # 2--3
    # connectivity = []
    # for ielem in np.arange(nelem):
    #     start = ielem * 4
    #     connectivity.append([start + 0, start + 1, start + 3, start + 2])

    # connectivity list, shared by all slices
    # with GLL_points_one_edge = [0, 2, 4] in the inparam.output.yaml,
    # the GLL point layout on each element is
    # 0--1--2
    # |  |  |
    # 3--4--5
    # |  |  |
    # 6--7--8
    connectivity = []
    for ielem in np.arange(nelem):
        start = ielem * 9
        connectivity.append([start + 0, start + 1, start + 4, start + 3])
        connectivity.append([start + 1, start + 2, start + 5, start + 4])
        connectivity.append([start + 3, start + 4, start + 7, start + 6])
        connectivity.append([start + 4, start + 5, start + 8, start + 7])

    # with GLL_points_one_edge = [0,1,2,3,4] in the inparam.output.yaml,
    # the GLL point layout on each element is
    # 0--1--2--3--4
    # |  |  |
    # 5--6--7--8--9
    # |  |  |
    # 10--11--12--13--14
    # 15--16--17--18--19
    # 20--21--22--23--24
    # connectivity = []
    # for ielem in np.arange(nelem):
    #     start = ielem * 25
    #     connectivity.append([start + 0, start + 1, start + 6, start + 5])
    #     connectivity.append([start + 1, start + 2, start + 7, start + 6])
    #     connectivity.append([start + 2, start + 3, start + 8, start + 7])
    #     connectivity.append([start + 3, start + 4, start + 9, start + 8])
    #     connectivity.append([start + 5, start + 6, start + 11, start + 10])
    #     connectivity.append([start + 6, start + 7, start + 12, start + 11])
    #     connectivity.append([start + 7, start + 8, start + 13, start + 12])
    #     connectivity.append([start + 8, start + 9, start + 14, start + 13])
    #     connectivity.append([start + 10, start + 11, start + 16, start + 15])
    #     connectivity.append([start + 11, start + 12, start + 17, start + 16])
    #     connectivity.append([start + 12, start + 13, start + 18, start + 17])
    #     connectivity.append([start + 13, start + 14, start + 19, start + 18])
    #     connectivity.append([start + 15, start + 16, start + 21, start + 20])
    #     connectivity.append([start + 16, start + 17, start + 22, start + 21])
    #     connectivity.append([start + 17, start + 18, start + 23, start + 22])
    #     connectivity.append([start + 18, start + 19, start + 24, start + 23])

    # loop over slices
    for islice, phi in enumerate(phi_slices[0:1]):
        # create vtk folder
        vtk_dir = data_dir + '/vtk_%s/slice%d' % (wave_channel, islice)
        os.makedirs(vtk_dir, exist_ok=True)

        # vtk mesh
        xyz = np.ndarray((nelem * ngll, 3))
        xyz[:, 0] = element_coords_sz[:, 0] * np.cos(phi)
        xyz[:, 1] = element_coords_sz[:, 0] * np.sin(phi)
        xyz[:, 2] = element_coords_sz[:, 1]
        vtk_mesh = pyvtk.UnstructuredGrid(list(zip(xyz[:, 0], xyz[:, 1], xyz[:, 2])),
                                        quad=connectivity)

        # loop over elements to read wave data
        wave = np.ndarray((nelem * ngll, ntime))
        for ielem in np.arange(nelem):
            wave[(ielem * ngll):(ielem * ngll + ngll), :] = dict_data_wave[nslice][ielem, islice, :, wave_dim, :]

        # loop over time to write vtk
        for itime in np.arange(ntime):
            vtk = pyvtk.VtkData(vtk_mesh, pyvtk.PointData(pyvtk.Scalars(wave[:, itime], name='U' + wave_channel)))
            vtk.tofile(vtk_dir + '/' + 'wave%d.vtk' % itime, 'binary')
            print('Done time step %d / %d' % (itime + 1, ntime), end='\r')
        print('\nDone slice %d' % (islice + 1))

        print(ModelName, ' wavefield value in ', [np.min(wave[:,-1]), np.max(wave[:,-1])])
        if np.max(wave[:,-1])>1e-3:
            print(ModelName, ' Attention !!!!')


Processing  SolidSphereModel0000RTZ
files to open:  ['axisem3d_synthetics.nc.rank4', 'axisem3d_synthetics.nc.rank14', 'axisem3d_synthetics.nc.rank23', 'axisem3d_synthetics.nc.rank22', 'axisem3d_synthetics.nc.rank5', 'axisem3d_synthetics.nc.rank0', 'axisem3d_synthetics.nc.rank18', 'axisem3d_synthetics.nc.rank12', 'axisem3d_synthetics.nc.rank16', 'axisem3d_synthetics.nc.rank1', 'axisem3d_synthetics.nc.rank10', 'axisem3d_synthetics.nc.rank11', 'axisem3d_synthetics.nc.rank3', 'axisem3d_synthetics.nc.rank19', 'axisem3d_synthetics.nc.rank15', 'axisem3d_synthetics.nc.rank20', 'axisem3d_synthetics.nc.rank9', 'axisem3d_synthetics.nc.rank7', 'axisem3d_synthetics.nc.rank13', 'axisem3d_synthetics.nc.rank17', 'axisem3d_synthetics.nc.rank2', 'axisem3d_synthetics.nc.rank21', 'axisem3d_synthetics.nc.rank6', 'axisem3d_synthetics.nc.rank8']
Done time step 128 / 128
Done slice 1
SolidSphereModel0000RTZ  wavefield value in  [-2.1164523786865175e-05, 3.2601048587821424e-05]


In [17]:
print('wavefield value in ', [np.min(wave[:,-1]), np.max(wave[:,-1])])

wavefield value in  [-8.323135930465853e-11, 5.0449568134158795e-11]


In [27]:
wave_dim

0